<img src="./pictures/logo_sizinglab.png" style="float:right; max-width: 60px; display: inline" alt="SizingLab" /></a>

# Battery and ESC selection
*Written by Marc Budinger (INSA Toulouse) and Scott Delbecq (ISAE-SUPAERO), Toulouse, France.*

## Design graph 

The following diagram represents the design graph of the battery and ESC selection.


![DesignGraph](pictures/ESCBatteryDesignGraph.png)

> **Questions:**
* Recall the main equations to take into account during ESC and battery selection
* Orientate the arrows
* Give equations order, inputs/outputs at each step of this part of sizing procedure



### Sizing code and optimization

> Exercice: propose a sizing code for the selection of a battery and ESC.


In [1]:
from utils.model_standard import CoreModel
from utils.model_serializer import ModelSerializer

In [2]:
import math
class BatteryEscModel(CoreModel):
    """
    Battery and ESC model class.
    ----------
    """
    def initialization(self):
        
        # Input variables
        self.add_input('U_bat_est', value=4.0, unit='V', comment='Battery voltage estimation')
        self.add_input('k_mb', value=2.0, unit='-', comment='ratio battery mass / Load mass')
        self.add_input('M_pay', value=10.0, unit='kg', comment='Payload mass')
        self.add_input('P_el_mot_hov', value=10.0, unit='W', comment='Electrical power consumption for one motor during hover')
        self.add_input('P_el_mot_to', value=10.0, unit='W', comment='Electrical power consumption for one motor during takeoff')
        self.add_input('U_mot_to', value=4.0, unit='V', comment='Motor voltage during takeoff')
        self.add_input('N_pro', value=4.0, unit='-', comment='Number of propellers')
        
        # Input parameters
        
        # Ref : MK-quadro
        M_bat_ref = .329 # [kg] mass
        E_bat_ref = 220.*3600.*.329 # [J]
        
        # Ref : Turnigy K_Force 70HV 
        P_esc_ref = 3108. # [W] Power
        M_esc_ref = .115 # [kg] Mass
        
        self.add_input('M_bat_ref', value=M_bat_ref, unit='kg', comment='Mass of the reference battery')
        self.add_input('E_bat_ref', value=E_bat_ref, unit='J', comment='Energy of the reference battery')
        self.add_input('M_esc_ref', value=M_esc_ref, unit='kg', comment='Mass of the reference ESC')
        self.add_input('P_esc_ref', value=P_esc_ref, unit='W', comment='Max power of the reference ESC')
        
        # Declare outputs
        self.add_output('N_s_bat', unit='-', comment='Number of battery cells')
        self.add_output('U_bat', unit='V', comment='Battery voltage')
        self.add_output('M_bat', unit='kg', comment='Battery mass')
        self.add_output('E_bat', unit='J', comment='Battery energy')
        self.add_output('C_bat', unit='A.s', comment='Capacity of the battery')
        self.add_output('I_bat', unit='A', comment='Current of the battery (hover flight)')
        self.add_output('t_hov', unit='min', comment='Hovering time')
        self.add_output('P_esc', unit='W', comment='Power electronic power (corner power or apparent power)')
        self.add_output('M_esc', unit='kg', comment='ESC maass')
        self.add_output('V_esc', unit='V', comment='ESC voltage')
        
    def computation_script(self):
        p = self.parameters
        
        p['N_s_bat'] = math.ceil(p['U_bat_est']/3.7) # [-] Cell number, round (up value)
        p['U_bat'] = 3.7 * p['N_s_bat'] # [V] Battery voltage
    
        p['M_bat'] = p['k_mb'] * p['M_pay'] # [kg] Battery mass
    
        # Hover --> autonomy
        p['E_bat'] = p['E_bat_ref'] * p['M_bat']/p['M_bat_ref']*.8 # [J] Energy  of the battery (.8 coefficient because 80% use only of the total capacity)
        p['C_bat'] = p['E_bat']/p['U_bat'] # [A.s] Capacity  of the battery 
        p['I_bat'] = (p['P_el_mot_hov'] * p['N_pro']) /.95/p['U_bat'] # [A] Current of the battery
        p['t_hov'] = p['C_bat']/p['I_bat']/60. # [min] Hover time 
        
        # ESC
        p['P_esc'] = p['P_el_mot_to'] * p['U_bat_est'] / p['U_mot_to'] # [W] power electronic power (corner power or apparent power)
        p['M_esc'] = p['M_esc_ref'] * (p['P_esc']/p['P_esc_ref']) # [kg] Mass ESC
        p['V_esc'] = 1.84 * p['P_esc']**0.36 # [V] ESC voltage        

In [3]:
battery_esc_model = BatteryEscModel('battery_esc')
battery_esc_model.initialization()

battery_esc_model.print_variables()

ms = ModelSerializer()
path = './models/'
file_name = 'battery_esc_model'
ms.save_model(battery_esc_model, path + file_name)

interactive(children=(Dropdown(description='Component', options=('battery_esc',), value='battery_esc'), Output…

In [4]:
inputs = {'P_el_mot_to': 25.0}

battery_esc_model.compute(inputs)

battery_esc_model.print_variables()

interactive(children=(Dropdown(description='Component', options=('battery_esc',), value='battery_esc'), Output…